**Library Imports**

In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import seaborn as sns

from scipy.stats import zscore

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

import tensorflow as tf

Panda Preferences

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Read CSV

In [3]:
df_train = pd.read_csv("../Datasets/dataset_andpre/EdgeIIot_train_smote.csv")
display(df_train)

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,bwd_header_size_tot,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,fwd_header_size_min_0,fwd_header_size_min_8,fwd_header_size_min_20,fwd_header_size_min_24,fwd_header_size_min_32,fwd_header_size_min_40,fwd_header_size_min_44,fwd_header_size_max_0,fwd_header_size_max_8,fwd_header_size_max_20,fwd_header_size_max_24,fwd_header_size_max_32,fwd_header_size_max_40,fwd_header_size_max_44,bwd_header_size_min_0,bwd_header_size_min_8,bwd_header_size_min_20,bwd_header_size_min_24,bwd_header_size_min_32,bwd_header_size_min_40,bwd_header_size_min_44,bwd_header_size_max_0,bwd_header_size_max_8,bwd_header_size_max_20,bwd_header_size_max_24,bwd_header_size_max_32,bwd_header_size_max_40,bwd_header_size_max_44,bwd_header_size_max_52,flow_FIN_flag_count_0,flow_FIN_flag_count_1,flow_FIN_flag_count_2,flow_FIN_flag_count_3,flow_FIN_flag_count_4,flow_FIN_flag_count_5,flow_FIN_flag_count_6,flow_FIN_flag_count_7,flow_SYN_flag_count_0,flow_SYN_flag_count_1,flow_SYN_flag_count_2,flow_SYN_flag_count_3,flow_SYN_flag_count_4,flow_SYN_flag_count_5,flow_SYN_flag_count_6,flow_SYN_flag_count_7,flow_SYN_flag_count_8,flow_SYN_flag_count_9,flow_SYN_flag_count_10,flow_RST_flag_count_0,flow_RST_flag_count_1,flow_RST_flag_count_2,flow_RST_flag_count_3,flow_RST_flag_count_4,history_originator_0,history_originator_1,history_originator_2,history_originator_3,history_originator_4,history_originator_5,history_originator_6,history_responder_0,history_responder_1,history_responder_2,history_responder_3,history_responder_4,history_responder_5,type
0,-0.014679,-0.179829,-0.005091,0.0,-0.044214,-0.223505,-0.756566,-0.651330,-0.014679,-0.044214,-0.756566,-0.184151,-0.046843,-0.778440,-0.777869,-0.778499,-1.303156,-0.563255,-0.589983,-0.184780,-0.040003,-0.499835,-0.071549,-0.224057,-0.170028,-0.202208,-0.218073,-0.026854,-0.0416,-0.042557,-0.034801,-0.029821,-0.033188,-0.224624,-0.170656,-0.208086,-0.221173,0.712756,0.273386,-0.002815,0.505192,-0.366493,-0.094334,-0.191905,-0.136594,-0.176039,-0.170188,0.700756,0.258094,-0.014705,0.494734,-0.371757,-0.016764,-0.166467,-1.229442,-0.211130,-0.033546,-0.004556,-0.003094,-0.004674,-0.003342,-0.002221,-0.001499,-0.210402,-0.211092,-0.207410,-0.057970,0.567019,0.279126,0.022051,0.428915,-0.303942,-0.801662,-0.708314,-0.725170,-0.653861,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,udp_flood
1,-0.405701,-0.179829,-0.005091,0

In [4]:
df_test = pd.read_csv("../Datasets/dataset_andpre/EdgeIIot_test.csv")
display(df_test)

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,bwd_header_size_tot,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,type,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,fwd_header_size_min_0,fwd_header_size_min_8,fwd_header_size_min_20,fwd_header_size_min_24,fwd_header_size_min_32,fwd_header_size_min_40,fwd_header_size_min_44,fwd_header_size_max_0,fwd_header_size_max_8,fwd_header_size_max_20,fwd_header_size_max_24,fwd_header_size_max_32,fwd_header_size_max_40,fwd_header_size_max_44,bwd_header_size_min_0,bwd_header_size_min_8,bwd_header_size_min_20,bwd_header_size_min_24,bwd_header_size_min_32,bwd_header_size_min_40,bwd_header_size_min_44,bwd_header_size_max_0,bwd_header_size_max_8,bwd_header_size_max_20,bwd_header_size_max_24,bwd_header_size_max_32,bwd_header_size_max_40,bwd_header_size_max_44,bwd_header_size_max_52,flow_FIN_flag_count_0,flow_FIN_flag_count_1,flow_FIN_flag_count_2,flow_FIN_flag_count_3,flow_FIN_flag_count_4,flow_FIN_flag_count_5,flow_FIN_flag_count_6,flow_FIN_flag_count_7,flow_SYN_flag_count_0,flow_SYN_flag_count_1,flow_SYN_flag_count_2,flow_SYN_flag_count_3,flow_SYN_flag_count_4,flow_SYN_flag_count_5,flow_SYN_flag_count_6,flow_SYN_flag_count_7,flow_SYN_flag_count_8,flow_SYN_flag_count_9,flow_SYN_flag_count_10,flow_RST_flag_count_0,flow_RST_flag_count_1,flow_RST_flag_count_2,flow_RST_flag_count_3,flow_RST_flag_count_4,history_originator_0,history_originator_1,history_originator_2,history_originator_3,history_originator_4,history_originator_5,history_originator_6,history_responder_0,history_responder_1,history_responder_2,history_responder_3,history_responder_4,history_responder_5
0,-0.540587,-0.179829,-0.005091,0,-0.302948,-0.211850,0.113379,0.010046,-0.540587,-0.302948,0.113379,-0.184151,-0.046843,1.467120,1.469549,1.468985,0.708010,-0.039559,-0.047807,-0.184780,-0.040003,-0.053936,-0.071549,-0.224057,-0.170028,-0.202208,-0.218073,-0.026854,-0.0416,-0.042557,-0.034801,-0.029821,-0.033188,-0.224624,-0.170656,-0.208086,-0.221173,-0.412900,-0.545421,-0.528133,-0.513173,-0.366493,-0.094334,-0.191905,-0.136594,-0.176039,-0.170188,-0.430596,-0.566738,-0.540613,-0.530154,-0.371757,-0.016764,-0.166467,0.638795,-0.211130,-0.033546,-0.004556,-0.003094,-0.004674,-0.003342,-0.002221,-0.001499,-0.210396,-0.211087,-0.207405,-0.05797,-0.477199,-0.536505,-0.510501,-0.528202,-0.303942,1.264714,-0.708314,1.400702,-0.653861,http_flood,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,-0.540588,-0.179829,-0.005091,0,-0.302948,-0.211850,0.113379,0.010046,-0.540588,-0.302948,0.113379,-0.184151,-0.046843,1.552666,1.555165,1.554603,0.708010,-0.039559,-0.047807,-0.184780,-0.040003,-0

---------------------------------------

**Create Model & Train Model**

In [5]:
df_train['is_attack'] = df_train['type'].apply(lambda x: 0 if x == "normal" else 1)
df_train.groupby('is_attack')['is_attack'].count()

is_attack
0     89090
1    801810
Name: is_attack, dtype: int64

In [6]:
df_test['is_attack'] = df_test['type'].apply(lambda x: 0 if x == "normal" else 1)
df_test.groupby('is_attack')['is_attack'].count()

is_attack
0     2310
1    87228
Name: is_attack, dtype: int64

In [7]:
df_train = df_train.drop('type', axis=1)
df_test = df_test.drop('type', axis=1)

In [8]:
# Split into input and output variables
x_columns = df_train.columns.drop('is_attack')
dummies = pd.get_dummies(df_train['is_attack'])
x = df_train[x_columns].values
attack = dummies.columns
y = dummies.values

In [9]:
# Split into input and output variables
x_columns_val = df_test.columns.drop('is_attack')
dummies_val = pd.get_dummies(df_test['is_attack'])
x_val = df_test[x_columns].values
y_val = dummies.values

In [10]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [11]:
y_test = np.argmax(y_test, axis=1)
y_train = np.argmax(y_train, axis=1)

In [12]:
x_test.shape,y_test.shape

((222725, 158), (222725,))

In [13]:
x_train.shape,y_train.shape

((668175, 158), (668175,))

In [14]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=x_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

# Define early stopping
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,mode="min",patience=10,verbose=1,min_lr=1e-7)
checkpoint = ModelCheckpoint('best_model_binary_smote.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=512, callbacks=[monitor, checkpoint])

Train on 668175 samples, validate on 222725 samples
Epoch 1/100
668175/668175 [==============================] - 6s 9us/sample - loss: 0.2870 - accuracy: 0.8856 - val_loss: 0.1881 - val_accuracy: 0.9174
Epoch 2/100
668175/668175 [==============================] - 4s 5us/sample - loss: 0.1913 - accuracy: 0.9182 - val_loss: 0.1794 - val_accuracy: 0.9182
Epoch 3/100
668175/668175 [==============================] - 4s 5us/sample - loss: 0.1846 - accuracy: 0.9195 - val_loss: 0.1780 - val_accuracy: 0.9183
Epoch 4/100
668175/668175 [==============================] - 3s 5us/sample - loss: 0.1821 - accuracy: 0.9202 - val_loss: 0.1779 - val_accuracy: 0.9185
Epoch 5/100
668175/668175 [==============================] - 3s 5us/sample - loss: 0.1808 - accuracy: 0.9204 - val_loss: 0.1742 - val_accuracy: 0.9224
Epoch 6/100
668175/668175 [==============================] - 3s 5us/sample - loss: 0.1793 - accuracy: 0.9207 - val_loss: 0.1737 - val_accuracy: 0.9223
Epoch 7/100
668175/668175 [===============

668175/668175 [==============================] - 3s 5us/sample - loss: 0.1730 - accuracy: 0.9229 - val_loss: 0.1699 - val_accuracy: 0.9228
Epoch 55/100
668175/668175 [==============================] - 4s 6us/sample - loss: 0.1723 - accuracy: 0.9239 - val_loss: 0.1699 - val_accuracy: 0.9250
Epoch 56/100
668175/668175 [==============================] - 5s 8us/sample - loss: 0.1733 - accuracy: 0.9227 - val_loss: 0.1702 - val_accuracy: 0.9239
Epoch 57/100
668160/668175 [============================>.] - ETA: 0s - loss: 0.1737 - accuracy: 0.9223
Epoch 00057: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
668175/668175 [==============================] - 4s 6us/sample - loss: 0.1737 - accuracy: 0.9223 - val_loss: 0.1700 - val_accuracy: 0.9240
Epoch 58/100
668175/668175 [==============================] - 4s 6us/sample - loss: 0.1729 - accuracy: 0.9229 - val_loss: 0.1687 - val_accuracy: 0.9248
Epoch 59/100
668175/668175 [==============================] - 4s 6us/sample - loss

In [15]:
# Load the best saved model
best_model = load_model('best_model_binary_smote.h5')

# Evaluate the best saved model
y_val = np.argmax(y_val, axis=1)
score = best_model.evaluate(x_val, y_val)
print('')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Input arrays should have the same number of samples as target arrays. Found 89538 input samples and 890900 target samples.

In [ ]:
model.summary()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 16))

ax.plot(history.history['loss'], label='train')
ax.plot(history.history['val_loss'], label='test')
ax.set_title('Model Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
pred = model.predict(x_val)

In [ ]:
def compute_metrics(pred, y_test):
    y_pred = np.round(pred).astype(int)
    
    correct = metrics.accuracy_score(y_test, y_pred)
    print(f"Accuracy: {correct}")
    
    recall = metrics.recall_score(y_test, y_pred, average = 'weighted')    
    print(f"Recall: {recall}")
       
    precision = metrics.precision_score(y_test, y_pred, average = 'weighted')
    print(f"Precision: {precision}")
    
    f1score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    print(f"F1Score: {f1score}")

In [ ]:
compute_metrics(pred, y_val)

-------------------------------------

**Result Plots**

In [ ]:
# Compute confusion matrix
y_pred = np.round(pred).astype(int)
cm = confusion_matrix(y_val, y_pred)

# Display confusion matrix
cmd = ConfusionMatrixDisplay(cm)
fig, ax = plt.subplots(figsize=(10, 10))
cmd.plot(ax=ax)

In [ ]:
# Usage of ExtraTreesClassifier for feature selection
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5, criterion ='entropy', max_features = 2)
extra_tree_forest.fit(x, y)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in  extra_tree_forest.estimators_], axis = 0)

In [ ]:
# Plor for the ExtraTreesClassifier output
plot.bar(x_columns, feature_importance_normalized)
plot.xlabel('Feature Labels')
plot.ylabel('Feature Importances')
plot.title('Comparison of different feature importances in the current dataset')
plot.xticks(rotation = 90)

# Plot size
plot.rcParams["figure.figsize"] = (70, 40)

plot.show()

## 